In [3]:
import pandas as pd 
basics = pd.read_csv("../Datasets/title.basics.tsv",sep = "\t",low_memory=True)
rating = pd.read_csv("../Datasets/title.ratings.tsv",sep= "\t",low_memory=True)
final_df = pd.merge(basics,rating,on="tconst")
final_df = final_df.dropna().reset_index(drop=True)
#Adding "BoxOfficeresult" column 
final_df["BoxOfficeResult"] = final_df.apply(lambda row: "Hit" if (row["averageRating"] >= 5 and row["numVotes"] >= 500 ) else "Flop",axis=1)
ffinal_df = final_df.dropna(axis = 0)
#Make startYear as int
ffinal_df['startYear'] = pd.to_numeric(ffinal_df['startYear'], errors='coerce')
ffinal_df = ffinal_df.dropna(subset=['startYear']).reset_index(drop=True)
#Create X
features = ["titleType","startYear","genres"]
X = ffinal_df[features] 
X.loc[:,"genres"] = X["genres"].replace("\\N", "Unknown")
from sklearn.preprocessing import MultiLabelBinarizer
# Split genres into lists
X.loc[:,"genres"] = X["genres"].apply(lambda x: [g.strip() for g in x.split(",")])
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(X["genres"])
# Convert to DataFrame
genre_encoded_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
# Merge back
X = pd.concat([X, genre_encoded_df], axis=1).drop(columns=["genres"])
# seperate the columns with non numeric values 
# manually set it
cat_cols = ["titleType"] 
num_cols = X.columns.drop(["titleType"])
#transformation
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder 
prepro = ColumnTransformer(transformers=[
    ("category",OneHotEncoder(handle_unknown="ignore"),cat_cols),
    ("num","passthrough",num_cols)
])
#gives numpy array
X_array = prepro.fit_transform(X)
# get names of features(columns)
feature_names = prepro.get_feature_names_out()
# into a df
X_final = pd.DataFrame(X_array, columns=feature_names).reset_index(drop = True)
# Transform Y 
from sklearn.preprocessing import LabelEncoder
y = ffinal_df.BoxOfficeResult 
label_encoder = LabelEncoder()
#gives numpy array
y_array = label_encoder.fit_transform(y)
#convert to a series
y_final = pd.Series(y_array,name="BoxOfficeResult").reset_index(drop=True)
# Seperate Data 
from sklearn.model_selection import train_test_split 
train_X , val_X , train_y , val_y = train_test_split(X_final,y_final,random_state=1)
#Oversample 
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=1)
X_train_new , y_train_new = sm.fit_resample(train_X,train_y)
#Create model
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(random_state=1)
model.fit(X_train_new,y_train_new)
predict = model.predict(val_X)
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(val_y,predict)
print("Accuracy : ", accuracy )
from sklearn.metrics import confusion_matrix
print(confusion_matrix(val_y, predict))
from sklearn.metrics import classification_report
print(classification_report(val_y, predict))

Accuracy :  0.7756937080240368
[[291357  80282]
 [ 11095  24642]]
              precision    recall  f1-score   support

           0       0.96      0.78      0.86    371639
           1       0.23      0.69      0.35     35737

    accuracy                           0.78    407376
   macro avg       0.60      0.74      0.61    407376
weighted avg       0.90      0.78      0.82    407376

